## LionAGI - 104

**Voting**

Quality is always hard to control for LLMs due to their inherent randomness. How can we assure our system always perform to our expectation?

for example, let's say we have 5 different generation of the same topic, with random configurations, how do we know which one is the best with having to read through all of them?

In [1]:
from IPython.display import display, Markdown


def printmd(string):
    display(Markdown(string))


openai_models = ["gpt-4o", "gpt-4o-mini", "gpt-4-turbo"]

config = {
    "provider": "openai",
    "api_key_schema": "OPENAI_API_KEY",
}

### Random Model

let us create a function to generate random intelligent models

In [2]:
import random
from lionagi import iModel, Branch


def generate_random_imodel(verbose=True):
    model = random.choice(openai_models)
    temperature = random.uniform(0.1, 1.0)
    top_p = random.uniform(0.7, 1.0)
    imodel = iModel(
        model=model,
        temperature=temperature,
        top_p=top_p,
        **config,
    )
    if verbose:
        printmd(
            f"Successfully generated a random Intelligent Model:\n\n{model},"
            f" \n\ntemperature: {temperature}, \n\ntop_p: {top_p}. \n\n ---"
        )
    return imodel

### Write explaination

now let us write a function to write a simple explaination

In [3]:
prompt = (
    "On balance sheet, we want to know equity vs liability "
    "vs assets. Accurately explain to a college freshman, around"
    " 200 words, organize in md style"
)

system_msg = (
    "you are a seasoned accountant, reply to prompt, no comments from you"
)

we will write a function to wrap the `Branch.chat` method inside

In [4]:
async def explain_balance_sheet(imodel, verbose=True):
    branch = Branch(system=system_msg, imodel=imodel)
    response = await branch.chat(instruction=prompt)
    if verbose:
        printmd(f"### Response: \n {response} \n --- \n")
    return response


model1 = generate_random_imodel()
response1 = await explain_balance_sheet(imodel=model1)

Successfully generated a random Intelligent Model:

gpt-4o-mini, 

temperature: 0.6599793885679598, 

top_p: 0.7572243238244627. 

 ---

### Response: 
 # Understanding Balance Sheet Components

A balance sheet is a financial statement that provides a snapshot of a company's financial position at a specific point in time. It consists of three main components: **assets**, **liabilities**, and **equity**.

## Assets
Assets are resources owned by the company that have economic value. They can be classified into two categories:
- **Current Assets**: These are assets that are expected to be converted into cash or used up within one year, such as cash, accounts receivable, and inventory.
- **Non-Current Assets**: These are long-term investments that will provide value over time, such as property, equipment, and intangible assets like patents.

## Liabilities
Liabilities represent the company's obligations or debts that it owes to outside parties. Like assets, they can also be classified:
- **Current Liabilities**: These are debts due within one year, such as accounts payable and short-term loans.
- **Long-Term Liabilities**: These are obligations that are due beyond one year, such as bonds payable and long-term loans.

## Equity
Equity, also known as shareholders' equity, represents the owners' claim on the assets after all liabilities have been settled. It includes common stock, retained earnings, and additional paid-in capital. 

The fundamental equation of a balance sheet is:
**Assets = Liabilities + Equity** 

This equation shows that what the company owns (assets) is financed either by borrowing (liabilities) or by the owners' investment (equity). 
 --- 


now let us generate 5 versions of these as our samples to select the best from. 

we will use a function called `alcall` to help us do asynchronous parallel API calls to LLMs.

In [5]:
from lionfuncs import alcall

models = [generate_random_imodel() for _ in range(5)]
samples = await alcall(models, explain_balance_sheet, verbose=False)

Successfully generated a random Intelligent Model:

gpt-4o-mini, 

temperature: 0.9688446743678979, 

top_p: 0.9325781413815237. 

 ---

Successfully generated a random Intelligent Model:

gpt-4-turbo, 

temperature: 0.8580176992323317, 

top_p: 0.9970266777779504. 

 ---

Successfully generated a random Intelligent Model:

gpt-4o, 

temperature: 0.8587724923453689, 

top_p: 0.8390517485163145. 

 ---

Successfully generated a random Intelligent Model:

gpt-4o-mini, 

temperature: 0.9673632222842298, 

top_p: 0.7915172546989881. 

 ---

Successfully generated a random Intelligent Model:

gpt-4o-mini, 

temperature: 0.707219117168443, 

top_p: 0.8052999894942017. 

 ---

let's take a look at one of them

In [6]:
printmd(samples[0])

# Understanding the Balance Sheet: Assets, Liabilities, and Equity

The balance sheet is a financial statement that provides a snapshot of a company's financial position at a specific point in time. It follows the fundamental accounting equation:

**Assets = Liabilities + Equity**

## Assets
Assets are what a company owns and can be classified into two main categories:

- **Current Assets**: These are short-term assets expected to be converted into cash within a year, like cash, inventory, and accounts receivable.
- **Non-Current Assets**: These are long-term assets that a company uses to generate revenue over time, such as property, plant, and equipment.

## Liabilities
Liabilities are what a company owes to others. They can also be divided into two types:

- **Current Liabilities**: Obligations that are due within one year, such as accounts payable and short-term loans.
- **Long-Term Liabilities**: These are obligations due in more than one year, such as bonds payable and long-term loans.

## Equity
Equity represents the ownership interest in the company. It is calculated as the difference between assets and liabilities. Equity can include:

- **Common Stock**: The amount invested by shareholders.
- **Retained Earnings**: Profits reinvested in the business rather than distributed as dividends.

In summary, the balance sheet provides vital information about a company's financial health, detailing how assets are financed through liabilities and equity.

---


## Rank

now we have the samples, it's time to invite a group of judge to vote for the best one. First we should instruct judge to respond in certain format

In [ ]:
propmt = "evaluate/score the quality of the response as if you are evaluating a PhD dissertation "
requested_fields = {"score": "integer 1-10", "reason": "string"}

In [7]:
from lionfuncs import as_readable_json


async def score_sample(sample):
    imodel = generate_random_imodel(verbose=False)
    branch = Branch(system="A harsh professor", imodel=imodel)
    responses = await branch.chat(
        instruction=prompt,
        context=sample,
        requested_fields=requested_fields,
    )
    return responses


response = await score_sample(samples[0])
printmd(as_readable_json(response))

{
    "score": 5,
    "reason": "The explanation provided in the dissertation lacks depth and critical analysis essential for a PhD level submission. While it accurately describes the basic components of a balance sheet, it fails to engage with more complex financial theories or provide any innovative perspectives that would demonstrate a doctoral level of understanding or contribution to the field of financial studies. Furthermore, the text could be improved by including examples, case studies, or recent research to support the explanations and enhance the academic rigor of the work."
}

now it is attempting to directly use the response as a dict
and average the score, however, we need to remember that

LLM responses are not always in the same format, so we need
to parse the data into proper data types

In [8]:
from lionfuncs import to_num, to_dict

In [9]:
def get_average_score(responses):
    scores = 0
    for res in responses:
        res = to_dict(res, fuzzy_parse=True)
        scores += to_num(res["score"])
    return scores / len(responses)

In [10]:
# now we are ready to score each sample with a group of judge LLM.


async def group_judge(idx, sample):
    responses = await alcall(
        [sample for _ in range(10)], score_sample
    )  # using 10 judge
    avg_score = get_average_score(responses=responses)
    print(f"Finished evaluating sample {idx}, score: {avg_score:.02f}")
    return {
        "idx": idx,
        "original_sample": sample,
        "response_content": responses,
        "score": avg_score,
    }

In [11]:
import asyncio


# let us run all judging together
async def rank_samples(samples):

    tasks = [
        asyncio.create_task(group_judge(idx, sample))
        for idx, sample in enumerate(samples)
    ]
    results = await asyncio.gather(*tasks)
    sorted_results = sorted(results, key=lambda x: x["score"], reverse=True)
    return sorted_results

In [12]:
final_results = await rank_samples(samples)

Finished evaluating sample 1, score: 6.90
Finished evaluating sample 3, score: 7.10
Finished evaluating sample 4, score: 7.60
Finished evaluating sample 0, score: 7.20
Finished evaluating sample 2, score: 6.60


In [13]:
# best

print(
    f"Sample No.{final_results[0]["idx"]}, Final Score: {final_results[0]["score"]}"
)
printmd(final_results[0]["original_sample"])

Sample No.4, Final Score: 7.6


# Understanding the Balance Sheet: Assets, Liabilities, and Equity

The balance sheet is a financial statement that provides a snapshot of a company's financial position at a specific point in time. It is based on the accounting equation:

**Assets = Liabilities + Equity**

## Assets
Assets are everything the company owns that has value. This includes cash, inventory, buildings, equipment, and accounts receivable. Assets are typically categorized as either current (to be used or converted into cash within a year) or non-current (long-term investments).

## Liabilities
Liabilities represent what the company owes to others. These are obligations that must be settled in the future, such as loans, accounts payable, and mortgages. Like assets, liabilities are divided into current (due within one year) and long-term (due in more than one year).

## Equity
Equity, often referred to as shareholders' equity or owner's equity, represents the owners' claim on the assets after all liabilities have been settled. It includes common stock, retained earnings, and additional paid-in capital. Essentially, equity reflects the net worth of the company.

In summary, the balance sheet illustrates the relationship between what a company owns (assets), what it owes (liabilities), and the residual interest of the owners (equity).

In [14]:
# worst

print(
    f"Sample No.{final_results[-1]["idx"]}, Final Score: {final_results[-1]["score"]}"
)
printmd(final_results[-1]["original_sample"])

Sample No.2, Final Score: 6.6


# Understanding the Balance Sheet: Equity, Liability, and Assets

A balance sheet is a financial statement that provides a snapshot of a company’s financial position at a specific point in time. It is structured around three main components: assets, liabilities, and equity.

## Assets
Assets represent everything the company owns and uses to generate revenue. They are typically classified into two categories:
- **Current Assets**: These are assets that are expected to be converted into cash or used up within one year, such as cash, accounts receivable, and inventory.
- **Non-current Assets**: These include long-term investments and property, plant, and equipment (PPE) that provide value over time.

## Liabilities
Liabilities are what the company owes to others. They are also divided into:
- **Current Liabilities**: These are obligations due within one year, such as accounts payable, short-term loans, and other short-term debts.
- **Non-current Liabilities**: These include long-term debts like bonds payable and long-term leases.

## Equity
Equity represents the owners' claims on the company after liabilities are settled. It includes:
- **Common Stock**: The value of shares issued to investors.
- **Retained Earnings**: Profits that have been reinvested in the business rather than distributed as dividends.

The fundamental accounting equation is:
\[ \text{Assets} = \text{Liabilities} + \text{Equity} \]
This equation must always balance, reflecting the financial stability of the company.